## Limpieza de archivo de despacho para liquidación

In [3]:
# Llamado de librerias
import pandas as pd

df = pd.read_excel('dspcttos/dspcttos_tx2.xlsx')
df.head(3)

,CONTRATO,VENDEDOR,COMPRADOR,TIPO,TIPOMERC,TIPO ASIGNA,DESP_HORA 01,DESP_HORA 02,DESP_HORA 03,DESP_HORA 04,...,TRF_HORA 16,TRF_HORA 17,TRF_HORA 18,TRF_HORA 19,TRF_HORA 20,TRF_HORA 21,TRF_HORA 22,TRF_HORA 23,TRF_HORA 24,fecha
0,69910,NESC,SFEC,PC,N,NB,2296.95,2296.95,2296.95,2296.95,...,313.541293,313.541293,313.541293,313.541293,313.541293,313.541293,313.541293,313.541293,313.541293,2024-10-01
1,70743,EPMG,SFEC,PC,N,NB,1500.00,1500.00,1500.00,1500.00,...,312.026656,312.026656,312.026656,312.026656,312.026656,312.026656,312.026656,312.026656,312.026656,2024-10-01
2,72555,SFEC,NEUC,PC,N,NB,2016.00,2016.00,2016.00,2016.00,...,353.297951,353.297951,353.297951,353.297951,353.297951,353.297951,353.297951,353.297951,353.297951,2024-10-01


In [6]:
# Eliminación de columnas que no son necesarrias para el despacho
df = df.drop(['TIPO','TIPOMERC','TIPO ASIGNA'],axis=1)
df.head(3)

,CONTRATO,VENDEDOR,COMPRADOR,DESP_HORA 01,DESP_HORA 02,DESP_HORA 03,DESP_HORA 04,DESP_HORA 05,DESP_HORA 06,DESP_HORA 07,...,TRF_HORA 16,TRF_HORA 17,TRF_HORA 18,TRF_HORA 19,TRF_HORA 20,TRF_HORA 21,TRF_HORA 22,TRF_HORA 23,TRF_HORA 24,fecha
0,69910,NESC,SFEC,2296.95,2296.95,2296.95,2296.95,2296.95,2296.95,2296.95,...,313.541293,313.541293,313.541293,313.541293,313.541293,313.541293,313.541293,313.541293,313.541293,2024-10-01
1,70743,EPMG,SFEC,1500.00,1500.00,1500.00,1500.00,1500.00,1500.00,1500.00,...,312.026656,312.026656,312.026656,312.026656,312.026656,312.026656,312.026656,312.026656,312.026656,2024-10-01
2,72555,SFEC,NEUC,2016.00,2016.00,2016.00,2016.00,2016.00,2016.00,2016.00,...,353.297951,353.297951,353.297951,353.297951,353.297951,353.297951,353.297951,353.297951,353.297951,2024-10-01
